---
title: "Chat GPT Text Generation"
description: "This notebook demonstrates how to use the Chat GPT model for text generation."
tags: ["NLP", "GPT", "Text Generation"]
---

In [ ]:
# Pre-requisites
!pip show openai || pip install openai
!pip show python-dotenv || pip install python-dotenv

In [ ]:
try:
    %load_ext dotenv
    print("dotenv extension loaded successfully.")
except Exception as e:
    print("Failed to load dotenv extension.")
    print("Error: ", str(e))

try:
    %dotenv
    print("Environment variables loaded successfully.")
except Exception as e:
    print("Failed to load environment variables.")
    print("Error: ", str(e))

try:
    %reload_ext dotenv
    print("dotenv extension reloaded successfully.")
except Exception as e:
    print("Failed to reload dotenv extension.")
    print("Error: ", str(e))

In [ ]:
from dotenv import load_dotenv
import os

# Load environment variables from .env file
if load_dotenv():
    print("Environment variables loaded successfully.")
else:
    print("Failed to load environment variables.")

# Get API key from environment variables
api_key = os.getenv('OPENAI_API_KEY')
if api_key:
    print("OPENAI_API_KEY loaded successfully.")
else:
    print("Failed to load OPENAI_API_KEY.")

# Get project ID from environment variables
project_id = os.getenv('PROJECT_ID')
if project_id:
    print("PROJECT_ID loaded successfully.")
else:
    print("Failed to load PROJECT_ID.")

# Get organization ID from environment variables
org_id = os.getenv('ORG_ID')
if org_id:
    print("ORG_ID loaded successfully.")
else:
    print("Failed to load ORG_ID.")

# Print the loaded values
print("OPENAI_API_KEY: ", api_key)
print("PROJECT_ID: ", project_id)
print("ORG_ID: ", org_id)

In [ ]:
from openai import OpenAI
import yaml
from datetime import datetime

client = OpenAI(
  organization='org-sKWn6Hj2qqyP19zHfpXM1j1N',
  project=os.getenv('PROJECT_ID'),
  api_key = os.getenv('OPENAI_API_KEY')
)

chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "Say this is a test",
        }
    ],
    model="gpt-3.5-turbo",
)

print(chat_completion.choices[0].message.content)

In [ ]:
def generate_content(prompt):
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {
                "role": "system",
                "content": prompt,
            },
        ],
    )
    # Get the content of the last message in the response
    return response.choices[0].message.content.strip()

In [ ]:
def create_assistant(client):
    name = input("Enter the name of the assistant: ")
    instructions = input("Enter the instructions for the assistant: ")
    assistant = client.beta.assistants.create(
        name=name,
        instructions=instructions,
        model="gpt-4o",
    )
    return assistant

# Call the function and store the result
assistant = create_assistant(client)
print(assistant.id)
# Your job is to take content and output a 4-word title that summarizes the content in a thought provoking manner. The title should be intriguing and attention getting for a reader. In other words, try to make it a worth while title for someone to be interested in.

In [ ]:
def create_message(content):
    thread = client.beta.threads.create()
    message = client.beta.threads.messages.create(
        thread_id=thread.id,
        role="user",
        content=content
    )
    return message


In [ ]:

prompt = input("Please enter your prompt for the GPT: ")
# Tell me a story about a cat named Nubi who smoked dubi's all day by the bay
# Tell me a story about a dog named Fido who loved to play fetch in the park
content = generate_content(prompt)
message = create_message(content)

In [ ]:
print(message.thread_id)
print(content)

In [ ]:
def create_and_poll_run(thread_id, assistant_id):
  run = client.beta.threads.runs.create_and_poll(
    thread_id=thread_id,
    assistant_id=assistant_id,
  )
  return run

thread_id = message.thread_id
assistant_id = assistant.id
run = create_and_poll_run(thread_id, assistant_id)


In [ ]:
if run.status == 'completed': 
  messages = client.beta.threads.messages.list(
    thread_id=message.thread_id
  )
  text_message = messages.data[0].content[0].text.value
  title = text_message
  print(title)
else:
  print(run.status)
  

In [ ]:
def create_jekyll_post(title, content):
    front_matter = {
        'title': title,
        'layout': 'journals',
        'date': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
        'categories': 'gpt',
    }
    post_content = f"---\n{yaml.dump(front_matter)}---\n{content}"
    filename = f"../_posts/{datetime.now().strftime('%Y-%m-%d')}-{title.lower().replace(' ', '-')}.md"
    os.makedirs(os.path.dirname(filename), exist_ok=True)
    with open(filename, 'w') as file:
        file.write(post_content)
    return filename

In [ ]:
file_path = create_jekyll_post(title, content)
print(f"The new file is created at: {file_path}")